
OpenAI Gym is a toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Pinball. For more information visit https://gym.openai.com.

Frozen Lake is an environment where the agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. For more information please visit https://gym.openai.com/envs/FrozenLake8x8-v0/.



In [6]:
from __future__ import print_function
from __future__ import division

In [19]:
import numpy as np
import gym
from gym import wrappers
import time
import scipy

def run_episode(env, policy, gamma, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma=gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma, epsilon=1e-20, max_iterations=100000):
    """
    This function implements value iteration algorithm for the infinite
    horizon discounted MDPs. If the sup norm of v_k - v_{k-1} is less than
    epsilon or number of iterations reaches max_iterations, it should return
    the value function.
    """
    start = time.time()
    v = np.zeros(env.nS)  # initialize value-function
    ########################### Your Code Here ###########################
    # Hint: see implementation of extract_policy
    
    for i in range(max_iterations):
        prev_v = np.copy(v)
        for s in range(env.nS):
            q_sa = [sum([p*(r + prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] 
            v[s] = max(q_sa)
        if (np.sum(np.fabs(prev_v - v)) <= epsilon):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    
                    
    ########################### End of your code #########################
    end = time.time()
    print("Value iteration took {0} seconds.".format(end - start))
    return v

if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_v = value_iteration(env, gamma);
        policy = extract_policy(optimal_v, gamma)
        policy_score = evaluate_policy(env, policy, gamma, n=1000)
        print('Average score = ', policy_score)

---------- Gamma=0.9 ----------
Value-iteration converged at iteration# 2357.
Value iteration took 1.3379542827606201 seconds.
Average score =  0.0022684173045983964
---------- Gamma=0.95 ----------
Value-iteration converged at iteration# 2357.
Value iteration took 1.3245046138763428 seconds.
Average score =  0.026259305305863505
---------- Gamma=0.99 ----------
Value-iteration converged at iteration# 2357.
Value iteration took 1.2950172424316406 seconds.
Average score =  0.3621868730565775
---------- Gamma=0.9999 ----------
Value-iteration converged at iteration# 2357.
Value iteration took 1.3328323364257812 seconds.
Average score =  0.8841686793884176
---------- Gamma=1 ----------
Value-iteration converged at iteration# 2357.
Value iteration took 1.308729887008667 seconds.
Average score =  0.866


Policy Iteration
---

In [17]:
def compute_policy_v(env, policy, gamma):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma, max_iterations=100000):
    """
    This function implements policy iteration algorithm.
    """
    start = time.time()
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    ########################### Your Code Here ###########################
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            end = time.time()
            print("Policy iteration took {0} seconds.".format(end - start))
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy
    
    
    ########################### End of your code #########################
    end = time.time()
    print("Policy iteration took {0} seconds.".format(end - start))
    return policy


if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_policy = policy_iteration(env, gamma=gamma)
        scores = evaluate_policy(env, optimal_policy, gamma=gamma)
        print('Average scores = ', np.mean(scores))

---------- Gamma=0.9 ----------
Policy iteration took 0.14729619026184082 seconds.
Policy-Iteration converged at step 5.
Average scores =  0.0065490249922309125
---------- Gamma=0.95 ----------
Policy iteration took 0.14989209175109863 seconds.
Policy-Iteration converged at step 3.
Average scores =  0.061114846132565896
---------- Gamma=0.99 ----------
Policy iteration took 0.8286290168762207 seconds.
Policy-Iteration converged at step 8.
Average scores =  0.4024744077259252
---------- Gamma=0.9999 ----------
Policy iteration took 2.636582851409912 seconds.
Policy-Iteration converged at step 12.
Average scores =  0.9008044318434361
---------- Gamma=1 ----------
Policy iteration took 1.6719274520874023 seconds.
Policy-Iteration converged at step 6.
Average scores =  0.87


We can see for finite action space, Policy Iteration algorithm is much faster and provide better score